# Домашнее задание по ML №2

## Предобработка

> **Задание**: Проведите предобработку текстов: если считаете нужным, выполните токенизацию, приведение к нижнему регистру, лемматизацию и/или стемминг. 
Ответьте на следующие вопросы:
1. Есть ли корреляция между средней длинной текста за день и DJIA?
2. Есть ли корреляция между количеством упоминаний Барака Обамы и США в день и DJIA? Учтите разные варианты написания США.
3. Каких статей больше: статей о России и Путине или об Исламском государстве (запрещенной законом РФ террористическая организации)?
4. О каких кризисах (crisis) пишут статьи?


Импортируем библиотеки. Для успешной работы с данными должны быть установлены следующие библиотеки:
* numpy
* pandas
* sklearn
* matplotlib
* nltk

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

Загружаем данные, смотрим на них:

In [35]:
df_news = pd.read_csv('Combined_News_DJIA.csv')

In [36]:
df_news.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

Теперь положим все топики в один столбец для удобства.

In [37]:
# функункция для избавления от b"..." в данных (оказавшихся там из-за неправильного сохранения?)
def beautify_string(s):
    return s[2:-1] if type(s) is not float else ''
    
# функция для объединения данных столбцов с топиками в один столбец
def merge_topics(sli):
    return ' '.join([beautify_string(s) for s in sli])

merge_topics(df_news.iloc[2, 2:27]) # смёрдженные топики второго столбца

'Remember that adorable 9-year-old who sang at the opening ceremonies? That was fake, too. Russia \'ends Georgia operation\' "If we had no sexual harassment we would have no children..." Al-Qa\'eda is losing support in Iraq because of a brutal crackdown on activities it regards as un-Islamic - including women buying cucumbers Ceasefire in Georgia: Putin Outmaneuvers the West Why Microsoft and Intel tried to kill the XO $100 laptop Stratfor: The Russo-Georgian War and the Balance of Power    I\'m Trying to Get a Sense of This Whole Georgia-Russia War: Vote Up If You Think Georgia Started It, Or Down If you Think Russia Did The US military was surprised by the timing and swiftness of the Russian military\'s move into South Ossetia and is still trying to sort out what happened, a US defense official said Monday U.S. Beats War Drum as Iran Dumps the Dollar Gorbachev: "Georgian military attacked the South Ossetian capital of Tskhinvali with multiple rocket launchers designed to devastate la

In [38]:
# итерируем строки датафрейма и склеиваем содержание нужных столбцов, запиываем результат в новый стоблец
df_news['topics'] = [merge_topics(df_news.iloc[row, 2:27]) for row in df_news.index]

In [39]:
df_news.topics[1]

'Why wont America and Nato help us? If they wont help us now, why did we help them in Iraq? Bush puts foot down on Georgian conflict Jewish Georgian minister: Thanks to Israeli training, we\'re fending off Russia  Georgian army flees in disarray as Russians advance - Gori abandoned to Russia without a shot fired Olympic opening ceremony fireworks \'faked\' What were the Mossad with fraudulent New Zealand Passports doing in Iraq? Russia angered by Israeli military sale to Georgia An American citizen living in S.Ossetia blames U.S. and Georgian leaders for the genocide of innocent people Welcome To World War IV! Now In High Definition! Georgia\'s move, a mistake of monumental proportions  Russia presses deeper into Georgia; U.S. says regime change is goal Abhinav Bindra wins first ever Individual Olympic Gold Medal for India  U.S. ship heads for Arctic to define territory Drivers in a Jerusalem taxi station threaten to quit rather than work for their new boss - an Arab The French Team is

Сейчас мы сделаем пробную предобработку текстов, чтобы продемонстрировать пример предобработки и ответить на вопросы из задания.

Стоп-слова:

In [40]:
from nltk.corpus import stopwords
stopset = stopwords.words('english')
stopset[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your']

Пунктуация:

In [41]:
from string import punctuation
punct = list(punctuation)
punct[:15]

['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/']

Токенизатор:

In [42]:
from nltk.tokenize import word_tokenize
word_tokenize('BREAKING: Musharraf to be impeached')

['BREAKING', ':', 'Musharraf', 'to', 'be', 'impeached']

Мы не будем лемматизировать заголовки новостей, потому что <s>этот лемматизатор в nltk как мёртвому припарка</s> <s>чувак-победитель на kaggle ничего не лемматизировал</s> информация о словоизменении может оказаться полезной.

Теперь определим общую функцию для предобработки:

In [43]:
def default_preprocessor(text):
    text = word_tokenize(text.lower()) # приводим к нижнему регистру и токенизируем
    return [w for w in text if w not in stopset + punct] # убираем стоп-слова и пунктуацию

In [44]:
default_preprocessor('BREAKING: Musharraf to be impeached')

['breaking', 'musharraf', 'impeached']

Результат предобработки поместим в _дублирующий, а не исходный,_ столбец, чтобы в дальнейшем иметь возможность поиграться с разными способами предобработки при векторизации:

In [45]:
df_news['preprocessed'] = df_news.topics.map(default_preprocessor)

In [46]:
df_news.preprocessed.head()

0    [georgia, 'downs, two, russian, warplanes, cou...
1    [wont, america, nato, help, us, wont, help, us...
2    [remember, adorable, 9-year-old, sang, opening...
3    [u.s., refuses, israel, weapons, attack, iran,...
4    [experts, admit, legalise, drugs, war, south, ...
Name: preprocessed, dtype: object

### Исследование данных

#### 1. Есть ли корреляция между средней длинной текста за день и DJIA?

мы воспользовались функцией, которая считает коэффициент корреляции Пирсона в модуле numpy https://docs.scipy.org/doc/numpy/reference/generated/numpy.corrcoef.html

In [47]:
# функция для объединения данных столбцов с топиками в один массив
def merge_topics_2arr(sli):
    arr = [beautify_string(s) for s in sli]
    return(arr)

# итерируем строки датафрейма и склеиваем содержание нужных столбцов, запиываем результат в новый стоблец
df_news['topics_list'] = [merge_topics_2arr(df_news.iloc[row, 2:27]) for row in df_news.index]

In [63]:
# посчитали среднюю длину текста за день
res = []
for topics in df_news['topics_list']:
    arr = [len(topic) for topic in topics]
    res.append(sum(arr)/len(arr))

In [64]:
# добавили в датафрейм новый столбец
df_news['av_len'] = res

In [72]:
# теперь посчитаем коэффициент корреляции Пирсона
# 1 способ
np.corrcoef(df_news['Label'], df_news['av_len'])[0,1]

-0.0027900870985365146

In [105]:
# 2 способ
from scipy.stats import pearsonr
x = df_news['Label']
y = df_news['av_len']
corr, p_value = pearsonr(x, y)

print('corr =', corr, 'p_value =', p_value )

corr = -0.00279008709854 p_value = 0.901034281785


Коэффициент корреляции Пирсона – это мера скоррелированности двух переменных. Он принимает значения от 1 до –1, где 1 означает, что корреляция между переменными идеальна, 0 – что корреляции нет, а –1, что имеется идеальная обратная корреляция.

В нашем случае результат близок к нулю, так что можно сказать, что корреляции между средней длинной текста за день и DJIA не наблюдается.

#### 2. Есть ли корреляция между количеством упоминаний Барака Обамы и США в день и DJIA? Учтите разные варианты написания США.

Будем считать, что одно+ упоминание имени барака обамы и/или сша в топике = одно упоминание в день

In [112]:
America = ['United States of America', 'America', 'US', 'USA', 'U.S.', 'Barack', 'Obama']

In [113]:
# подсчитаем, в скольких топиках за день упоминается америка или барак обама
def is_x(arr):
    is_x = []
    for topics in df_news['topics_list']:
        n = 0
        for topic in topics:
            i = 0
            for word in arr:
                if word in topic:
                    i += 1
            if i>0:
                n += 1
        is_x.append(n)
    return(is_x)

In [114]:
df_news['is_america'] = is_x(America)

In [118]:
# теперь посчитаем коэффициент корреляции Пирсона
# 1 способ
np.corrcoef(df_news['is_america'], df_news['Label'])[0,1]

-3.1548429719110349e-05

In [119]:
# 2 способ
from scipy.stats import pearsonr
x = df_news['Label']
y = df_news['is_america']
corr, p_value = pearsonr(x, y)

print('corr =', corr, 'p_value =', p_value )

corr = -3.15484297191e-05 p_value = 0.998878079838


ну я даже посчитала двумя способами, но не понимаю, почему там -3..

#### 3. Каких статей больше: статей о России и Путине или об Исламском государстве (запрещенной законом РФ террористическая организации)?

In [120]:
Russia = ['Russian Federation', 'Russia', 'RF', 'Putin']
ISIL = ['Islamic State', 'ISIL', 'ISIS', 'IS', 'Daesh']

In [121]:
df_news['is_russia'] = is_x(Russia)
df_news['is_isil'] = is_x(ISIL)

In [125]:
print('russia =', sum(df_news['is_russia']), 'ISIL =', sum(df_news['is_isil']))

russia = 2054 ISIL = 1017


Про Россию и Путина статей больше.

#### 4. О каких кризисах (crisis) пишут статьи?

In [144]:
import re
crs = []
for topics in df_news['topics_list']:
    for topic in topics:
        if 'crisis' in topic:
            cr = re.findall('[a-zA-Z]+ crisis', topic)
            for i in cr:
                crs.append(i)

In [151]:
print(len(crs), len(set(crs)))

383 134


In [159]:
d = {}
for cr in crs:
    if cr not in d:
        d[cr] = 1
    else:
        d[cr] += 1

In [161]:
for slovo in sorted(d, key = d.get, reverse = True):
    print(slovo + ' - ' + str(d[slovo]))

financial crisis - 40
economic crisis - 20
the crisis - 19
debt crisis - 18
food crisis - 16
raine crisis - 14
Ukraine crisis - 11
nuclear crisis - 10
humanitarian crisis - 8
political crisis - 8
refugee crisis - 8
banking crisis - 7
ria crisis - 6
a crisis - 6
s crisis - 6
in crisis - 6
euro crisis - 6
as crisis - 6
grant crisis - 4
fugee crisis - 4
of crisis - 4
water crisis - 3
Syria crisis - 3
bank crisis - 3
aq crisis - 3
migrant crisis - 3
health crisis - 3
currency crisis - 3
global crisis - 3
Greek crisis - 3
Migrant crisis - 3
Korea crisis - 3
eek crisis - 3
grants crisis - 2
climate crisis - 2
pan crisis - 2
The crisis - 2
to crisis - 2
ola crisis - 2
oil crisis - 2
eurozone crisis - 2
fa crisis - 2
gration crisis - 2
Lanka crisis - 2
Ebola crisis - 2
diplomatic crisis - 2
energy crisis - 2
men crisis - 2
Iraq crisis - 2
za crisis - 2
housing crisis - 2
reached crisis - 2
rrency crisis - 1
legal crisis - 1
treatment crisis - 1
superbug crisis - 1
word crisis - 1
radiation cri

Как видно из представленного частотного списка, самые обсуждаемые -- это financial crisis и economic crisis

## Обучение классификаторов

> **Задание**: Вам предстоит решить следующую задачу: по текстам новостей за день определить, вырастет или понизится DJIA. То есть, метки класса (y) заданы DJIA, признаки (X) требуется извлечь из текстов.

> Обучающее и тестовое множество строится так: данные до начала 2015 года используются для обучения, данные с 2015 года и позже – для тестирования.

> Используйте любой известный вам алгоритм классификации текстов для того,  Используйте $tf-idf$ преобразование, сингулярное разложение, нормировку признакого пространства и любые другие техники обработки данных, которые вы считаете нужным. Используйте accuracy и F-measure  для оценки качества классификации. Покажите, как  $tf-idf$ преобразование или сингулярное разложение или любая другая использованная вами техника влияет на качество классификации. 

> Если у выбранного вами алгоритма есть гиперпараметры (например, alpha в преобразовании Лапласа для метода наивного Байеса), покажите, как изменение гиперпараметра влияет на качество классификации. 

Разделяем выборку на train и test, как сказано в задании:

In [222]:
train = df_news[df_news['Date'] < '2015-01-01']
test = df_news[df_news['Date'] > '2014-12-31']

In [42]:
print(train.iloc[0]['Date'] + ', ' + train.iloc[-1]['Date'])

2008-08-08, 2014-12-31


In [43]:
print(test.iloc[0]['Date'], test.iloc[-1]['Date'])

2015-01-02 2016-07-01


Попробуем сначала деревья решений, CountVectorizer и наш дефолтный препроцессинг:

In [229]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [249]:
ppl = Pipeline(
    [('vect', CountVectorizer(tokenizer=default_preprocessor)),
    ('tree', DecisionTreeClassifier(random_state=42))]
)
ppl.fit(train.topics, y=train.Label)

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...plit=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=42, splitter='best'))])

In [251]:
print(classification_report(test.Label, ppl.predict(test.topics)))
print(accuracy_score(test.Label, ppl.predict(test.topics)))

             precision    recall  f1-score   support

          0       0.51      0.49      0.50       186
          1       0.53      0.55      0.54       192

avg / total       0.52      0.52      0.52       378

0.518518518519


Как мы видим, дефолтный вариант работает очень плохо: результаты практически не отличаются от подбрасывания монетки.

Попробуем немного поиграться с параметрами: зададим минимальную/максимальную частоту документа, другой ngram_range, максимальное количество листьев...

In [301]:
ppl = Pipeline(
    [('vect', CountVectorizer(tokenizer=default_preprocessor, min_df=5)),
    ('tree', DecisionTreeClassifier(random_state=42))]
)
ppl.fit(train.topics, y=train.Label)

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...plit=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=42, splitter='best'))])

In [302]:
print(classification_report(test.Label, ppl.predict(test.topics)))
print(accuracy_score(test.Label, ppl.predict(test.topics)))

             precision    recall  f1-score   support

          0       0.54      0.47      0.51       186
          1       0.55      0.61      0.58       192

avg / total       0.54      0.54      0.54       378

0.544973544974


Уже немного лучше! Немного поэксперементировав, поняли, что оптимальная минимальная частота — 5, а вот максимальную лучше не трогать. Попытки подобрать максимальное количество листьев и оптимальный разброс нграммов успехом не увенчались.

А теперь попробуем использовать TfidfVectorizer вместо CountVectorizer.

In [341]:
ppl = Pipeline(
    [('vect', TfidfVectorizer(tokenizer=default_preprocessor)),
    ('tree', DecisionTreeClassifier(random_state=45))]
)
ppl.fit(train.topics, y=train.Label)
print(classification_report(test.Label, ppl.predict(test.topics)))
print(accuracy_score(test.Label, ppl.predict(test.topics)))

             precision    recall  f1-score   support

          0       0.52      0.48      0.50       186
          1       0.53      0.57      0.55       192

avg / total       0.53      0.53      0.53       378

0.526455026455


Ура, $tf-idf$ преобразование делает вещи хуже! (Проверено на random_state от 35 до 45 и с разными значениями min_df и max_df, вверху — лучший результат, которого получилось добиться).

Вернёмся к обычному CountVectorizer и теперь попробуем другой препроцессинг. Например, попробуем не убирать стоп-слова (а только пунктуацию).

In [349]:
ppl = Pipeline(
    [('vect', CountVectorizer(tokenizer=word_tokenize, stop_words=punct)),
    ('tree', DecisionTreeClassifier(random_state=42))]
)
ppl.fit(train.topics, y=train.Label)
print(classification_report(test.Label, ppl.predict(test.topics)))
print(accuracy_score(test.Label, ppl.predict(test.topics)))

             precision    recall  f1-score   support

          0       0.47      0.42      0.44       186
          1       0.49      0.54      0.51       192

avg / total       0.48      0.48      0.48       378

0.481481481481


Нет, лучше не становится. (Тоже проверено при разных параметрах).

А теперь попробуем использовать лемматизатор<s> и понять, почему не надо этого делать</s>:

In [336]:
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

In [26]:
wnl.lemmatize('added')

'added'

In [27]:
wnl.lemmatize('kittens')

'kitten'

In [28]:
wnl.lemmatize('longer')

'longer'

Как мы видим, лемматизируются только существительные, всё остальное не лемматизируется. Попробуем всё же применить этот лемматизатор в действии.

In [355]:
def lemm_preproc(text):
    text = wnl.lemmatize(text)
    return default_preprocessor(text)
lemm_preproc(train.topics[0][:50])

['georgia', "'downs", 'two', 'russian', 'warplanes', 'countries']

In [364]:
ppl = Pipeline(
    [('vect', CountVectorizer(tokenizer=lemm_preproc, min_df=5)),
    ('tree', DecisionTreeClassifier(random_state=42))]
)
ppl.fit(train.topics, y=train.Label)
print(classification_report(test.Label, ppl.predict(test.topics)))
print(accuracy_score(test.Label, ppl.predict(test.topics)))

             precision    recall  f1-score   support

          0       0.54      0.47      0.51       186
          1       0.55      0.61      0.58       192

avg / total       0.54      0.54      0.54       378

0.544973544974


Ожидаемо, результат особенно не изменился (Лучший результат из min_df от 1 до 12, random_state от 35 до 45).

Попробуем теперь какую-нибудь другую модель, например, логистическую регрессию.

In [369]:
from sklearn.linear_model import LogisticRegression
LogisticRegression()

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [372]:
ppl = Pipeline(
    [('vect', CountVectorizer(tokenizer=default_preprocessor, min_df=5)),
    ('tree', LogisticRegression(random_state=42))]
)
ppl.fit(train.topics, y=train.Label)
print(classification_report(test.Label, ppl.predict(test.topics)))
print(accuracy_score(test.Label, ppl.predict(test.topics)))

             precision    recall  f1-score   support

          0       0.40      0.33      0.36       186
          1       0.44      0.52      0.48       192

avg / total       0.42      0.43      0.42       378

0.425925925926
